# Data Analysis and Mining
## Linear Regression Tutorial



### Objectives
This tutorial explains how to:
- Perform simple linear regression
- Use correlation analysis to determine whether two quantities are related to justify fitting the data.
- Fit a linear model to the data.
- Evaluate the goodness of fit by plotting residuals and looking for patterns.
- Calculate measures of goodness of fit R2 and adjusted R2

## Import dataset

In [ ]:
import pandas as pd

# Specify the path to the dataset CSV file
dataset_path = "../Dataset/dataset.csv"

# Read the dataset into a pandas DataFrame
df = pd.read_csv(dataset_path)
